<a href="https://colab.research.google.com/github/anmolag190153/BCS_summer_project_SER/blob/main/SER_MLP_model_k_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,regularizers,models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import CSVLogger

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/RAVDESS_dataset'
k = 0
x_data = np.ndarray(shape = (24 * 60, 120000))
y_data = np.ndarray(shape = (24 * 60, 1))
for i,actor in enumerate(os.listdir(path)):
    path1 = os.path.join(path,actor)
    for j, file in enumerate(os.listdir(path1)):
        b=file[6:8]
        y_data[k,0] = b
        y_data[k,0] = y_data[k, 0] - 1
        temp_path = os.path.join(path1, file)
        # print(temp_path)
        var, _ = librosa.load(temp_path)
        x_data[k, :var.shape[0]] = var[:]
        x_data[k,var.shape[0]:] = 0
        k = k + 1

In [ ]:
sr=22100

In [ ]:
def build_fc_model(input_shape=(180,1), num_class=8):
    
    
    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-2)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dense(64,activation='relu'),
                                    tf.keras.layers.Dense(num_class,activation='softmax')
                                    ])

    return model

In [ ]:
def extract_feature(file):
    result=np.array([])
    mfcc=np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=40).T,axis=0)
    result=np.hstack((result,mfcc))
    chroma=np.mean(librosa.feature.chroma_stft(file,sr=sr).T,axis=0)
    result=np.hstack((result,chroma))
    mel=np.mean(librosa.feature.melspectrogram(file,sr=sr).T,axis=0)
    result=np.hstack((result,mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr=sr).T,axis=0)
    result=np.hstack((result,spectral_rolloff))
    return result

In [ ]:
x=[]
for i in range(24*60):
    feature = extract_feature(x_data[i,:])
    x.append(feature)
#x_train, x_test, y_train, y_test = train_test_split(np.array(x), y_data, test_size = 0.2, random_state = 9)

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
print(x.shape, y_data.shape)

(1440, 181) (1440, 1)


In [ ]:
accuracy_perfold = []
loss_perfold = []
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits = 10)
for train_index, test_index in folds.split(x, y_data):
    X_train, X_test, y_train, y_test = x[train_index], x[test_index], \
                                       y_data[train_index], y_data[test_index]
    model = build_fc_model()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test))
    score  = model.evaluate(X_test, y_test)
    accuracy_perfold.append(score[1] * 100)
    loss_perfold.append(score[0]) 

Epoch 1/10
41/41 [==============================] - 1s 13ms/step - loss: 4.3655 - accuracy: 0.2778 - val_loss: 3.6667 - val_accuracy: 0.3542
Epoch 2/10
41/41 [==============================] - 0s 9ms/step - loss: 3.1062 - accuracy: 0.4437 - val_loss: 2.9274 - val_accuracy: 0.4236
Epoch 3/10
41/41 [==============================] - 0s 9ms/step - loss: 2.5570 - accuracy: 0.5201 - val_loss: 2.5333 - val_accuracy: 0.4514
Epoch 4/10
41/41 [==============================] - 0s 9ms/step - loss: 2.1628 - accuracy: 0.5594 - val_loss: 2.4464 - val_accuracy: 0.4097
Epoch 5/10
41/41 [==============================] - 0s 8ms/step - loss: 1.9159 - accuracy: 0.6080 - val_loss: 2.4907 - val_accuracy: 0.4236
Epoch 6/10
41/41 [==============================] - 0s 9ms/step - loss: 1.7944 - accuracy: 0.6157 - val_loss: 2.4434 - val_accuracy: 0.4097
Epoch 7/10
41/41 [==============================] - 0s 9ms/step - loss: 1.6778 - accuracy: 0.6296 - val_loss: 2.1927 - val_accuracy: 0.4375
Epoch 8/10
41/41 [=

In [ ]:
result = np.average(accuracy_perfold)
print('Accuracy obtained on validation is ',result,'%')

Accuracy obtained on validation is  43.81944477558136 %
